In [1]:
#%pip install git+https://github.com/huggingface/peft
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl 
#%pip install https://pypi.org/simple/ bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
!rm -rf /kaggle/working/*

In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer ,BitsAndBytesConfig
import torch

### 4bit Nested

In [4]:
# %%time

# bnb_config = BitsAndBytesConfig(  
#     load_in_4bit=True,
#     bnb_4bit_quant_type= "nf4", # normal float : a new 4bit datatype adapted for weights that have been initialized using a normal distribution. 
#     bnb_4bit_compute_dtype= torch.bfloat16, # Set the computation data type to bfloat16 instead of default float32
#     bnb_4bit_use_double_quant= True, # Nested quantization : where the quantization constants from the first quantization are quantized again. - no performance lost
# )

# config = PeftConfig.from_pretrained("FO-UA/adapt-llm-ghost-Fr-80xr512")
# model_new = AutoModelForCausalLM.from_pretrained("AdaptLLM/finance-chat", device_map="cuda", quantization_config=bnb_config)
# model_new = PeftModel.from_pretrained(model_new, "FO-UA/adapt-llm-ghost-Fr-80xr512") #.to("cuda")

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained("FO-UA/adapt-llm-ghost-Fr-80xr512", trust_remote_code=True)
# tokenizer.padding_side = 'right'
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_eos_token = True
# tokenizer.add_bos_token, tokenizer.add_eos_token

### 4bit + TorchCompile

In [5]:
%%time
bnb_config = BitsAndBytesConfig(  
    load_in_4bit=True,
    bnb_4bit_quant_type= "nf4", # normal float : a new 4bit datatype adapted for weights that have been initialized using a normal distribution. 
    bnb_4bit_compute_dtype= torch.bfloat16, # Set the computation data type to bfloat16 instead of default float32
    bnb_4bit_use_double_quant= False, # Nested quantization : where the quantization constants from the first quantization are quantized again. - no performance lost
)

config = PeftConfig.from_pretrained("FO-UA/adapt-llm-ghost-Fr-80xr512")
model_new_4b = AutoModelForCausalLM.from_pretrained("AdaptLLM/finance-chat", device_map="cuda", quantization_config=bnb_config)
model_new_4b = PeftModel.from_pretrained(model_new_4b, "FO-UA/adapt-llm-ghost-Fr-80xr512") #.to("cuda")
model_new_compile = torch.compile(model_new_4b)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("FO-UA/adapt-llm-ghost-Fr-80xr512", trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

adapter_config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

CPU times: user 1min 2s, sys: 1min 10s, total: 2min 13s
Wall time: 4min 43s


(True, True)

# List input : Input Preparation 

In [6]:
import pandas as pd

def formatList(L):
    L_res=[]
    i=0

    while i < len(L) : 
        strr="{ " + L[i] + " },"
        j=i+1
        while (j< len(L)) and (L[i].split(",")[0]== L[j].split(",")[0]): 
            strr+= "{ " + L[j].split('"Explanation":')[0] + " },"
            j+=1
           
        L_res.append(strr + ' "Explanation":' + L[j-1].split('"Explanation":')[1])
        i=j
        
    return L_res

data=pd.read_csv("/kaggle/input/ghost-worker-tris/Fraud_ghost_test_full_24_anonym.csv")

json_data = data.to_json(orient='records')

L=json_data.split('},{')

L[0]=L[0][2:]
L[-1]=L[-1][:-2]

L_test_format = formatList(L)

for i in range(len(L_test_format)) : 
    Scenario = L_test_format[i].split('"Explanation":"')
    L_test_format[i] = Scenario[0] + '\n "Explanation":"' + Scenario[1]
    
## Big List

data=pd.read_csv("/kaggle/input/ghost-worker-tris/Fraud_ghost_full_91_anonym.csv")

json_data = data.to_json(orient='records')

L=json_data.split('},{')

L[0]=L[0][2:]
L[-1]=L[-1][:-2]

L_test_format_2 = formatList(L)

for i in range(len(L_test_format_2)) : 
    Scenario = L_test_format_2[i].split('"Explanation":"')
    L_test_format_2[i] = Scenario[0] + '\n "Explanation":"' + Scenario[1]

L_Full = L_test_format + L_test_format_2
len(L_Full)

29

# Batch function Definition & Generation

## V1

In [7]:
# def batch_generation(L,modell) :
    
#     L_result=[]
#     for i in range(len(L)) :
#         user_input = (f'''{L[i]} 
        
#             Instruction : Generate a Ghost Worker Fraud scenario :           
#             ''' +'''
            
#                         {
#               "PersonID": "",
#               "Document_Type": "",
#               "Document_name": "",
#               "Documents_Deposit": "",
#               "Deposit_Platform": "",
#               "Absence_CODE": "",
#               "Absence_Reason": "",
#               "Seniority_in_Group (in days)": "",
#               "Leave_Date": "",
#               "Physical_Leave": "",
#               "Entry_Category": "",
#               "Entry_Date": "",
#               "Qualification": "",
#               "CODENT": "",
#               "CODSOR": "",
#               "Current_company": "",
#               "Leave_Reason": "",
#               "Transfer_Reason": "",
#               "Transfer_company": "",
#               "Explanation": ""
#             }


#         '' Show the generated Fraud scenario, in json format.
#     ''')
        
#         initial_prompt = ''' 
#         Note that the actual year we are currently in is : 2024. 
#         I want it to be in json format.

#         The data example below is a database extract of historical data of the same employee, it shows his entry and leave history in the Enterprises group,his seniority, leave reasons, the documents he deposited if he did, etc. 
#         Therefore, we can detect a ghost fraud scenario, through analyzing multiple rows of one employee, and trying to find inconsistencies, like absence of necessay documents, unconventional absence rates and reasons, etc.
#         You will be a ghost worker Fraud Scenrios Generator. You will Generate Logical and reasonable Ghost-worker Fraud Scenarios in json format, featuring the Keys in the example down below.

#         Documents_Deposit represents the status of the documents deposit.
#         Deposit_Platform represents the platform on which the documents have been deposited, for example H or F, or O for Onboarding deposit.

#         Leave_Date, Physical_Leave Take a default value for employees who are currentluy in the company and have not left yet, this value is usually '31/12/2050'
#         '''

#         prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"

#         inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(modell.device) #to("cuda") #.to(model_new.device)
#         outputs =modell.generate(input_ids=inputs, 
#                                  max_new_tokens=3000,  #1200 for 3 examples 
#                                  do_sample=True,
#                                  temperature=1.2
#                                 )[0]

#         answer_start = int(inputs.shape[-1])
#         pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)
#         L_result.append(pred)
        
#         # save str in file format
#         print(user_input)
#         file_name = 'Assis_55_Temp12_Ghost_correct' + str(i) + '.txt'
#         with open( file_name , 'w') as f:
#             f.write(pred)
#         print(f' \n file {i} saved')
        
#         print(pred)
#         print('\n ------------------')
    
#     return L_result


## V2

In [8]:
def batch_generation(L,modell) :
    
    L_result=[]
    for i in range(len(L)) :
        user_input = (f'''{L[i]} 
        
           Instruction : Generate a Ghost Worker Fraud scenario :           
            ''' +'''
            
                        {
              "PersonID": "",
              "Document_Type": "",
              "Document_name": "",
              "Documents_Deposit": "",
              "Deposit_Platform": "",
              "Absence_CODE": "",
              "Absence_Reason": "",
              "Seniority_in_Group (in days)": "",
              "Leave_Date": "",
              "Physical_Leave": "",
              "Entry_Category": "",
              "Entry_Date": "",
              "Qualification": "",
              "CODENT": "",
              "CODSOR": "",
              "Current_company": "",
              "Leave_Reason": "",
              "Transfer_Reason": "",
              "Transfer_company": "",
              "Explanation": ""
            }

        '' Show the generated Fraud scenario, in json format.
    
    ''')
        initial_prompt = ''' 
        Note that the actual year we are currently in is : 2024. 
        I want it to be in json format.

        The data example below is a database extract of historical data of the same employee, it shows his entry and leave history in the Enterprises group,his seniority, leave reasons, the documents he deposited if he did, etc. 
        Therefore, we can detect a ghost fraud scenario, through analyzing multiple rows of one employee, and trying to find inconsistencies, like absence of necessay documents, unconventional absence rates and reasons, etc.
        You will be a ghost worker Fraud Scenrios Generator. You will Generate Logical and reasonable Ghost-worker Fraud Scenarios in json format, featuring the Keys in the example down below.

        Trnasfer company can only have values from this list : ['TWW', 'ATX', 'TMG', 'EXM', 'IIS', 'IMI', 'ITI', 'ITM', 'SMS', 'TW2', 'SBS']
        Original company can only have values from this list : ['AFA', 'IIS', 'PRO', 'ITM', 'EUR', 'SMA', 'SMS', 'SBS', 'IMI', 'TWW', 'TMG', 'ATX', 'EXM', 'ITI']

        Documents_Deposit : the status of the documents deposit.
        Deposit_Platform represents the platform on which the documents have been deposited, for example H or F, or O for Onboarding deposit.

        Leave_Date, Physical_Leave Take a default value for employees who are currentluy in the company and have not left yet, this value is usually '31/12/2050'
        '''

        prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"

        inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(modell.device) #to("cuda") #.to(model_new.device)
        outputs =modell.generate(input_ids=inputs, 
                                 max_new_tokens=3000,  #1200 for 3 examples 
                                 do_sample=True,
                                 temperature=1.3
                                )[0]

        answer_start = int(inputs.shape[-1])
        pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)
        L_result.append(pred)
        
        # save str in file format
        print(user_input)
        file_name = 'Assis_55_Temp13_Ghost_correct' + str(i) + '.txt'
        with open( file_name , 'w') as f:
            f.write(pred)
        print(f' \n file {i} saved')
        
        # print(pred)
        print('\n ------------------')
    
    return L_result


In [ ]:
batch_generation(L_Full,model_new_compile)